# Setup

In [201]:
# OctoAI
# ! pip install langchain langchain-community faiss-cpu sentence-transformers octoai-sdk langchain-text-splitters lxml tiktoken python-dotenv 'arize-phoenix[evals]'

In [202]:
import octoai
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IjNkMjMzOTQ5In0.eyJzdWIiOiJkNTY1Y2Q3YS0zYmNjLTQzNDgtOGQxYy1mMGY0ZjY0ODkyYzciLCJ0eXBlIjoidXNlckFjY2Vzc1Rva2VuIiwidGVuYW50SWQiOiJkMzRmOGVkZC1kMzE1LTQ4NTktOTc0Zi03MjJiMzNlNDA5ZDIiLCJ1c2VySWQiOiJhZTMxM2ExZS1lYTI3LTRkOTItYjk5OS1iOTNlY2Q0YjQ3NTgiLCJhcHBsaWNhdGlvbklkIjoiYTkyNmZlYmQtMjFlYS00ODdiLTg1ZjUtMzQ5NDA5N2VjODMzIiwicm9sZXMiOlsiRkVUQ0gtUk9MRVMtQlktQVBJIl0sInBlcm1pc3Npb25zIjpbIkZFVENILVBFUk1JU1NJT05TLUJZLUFQSSJdLCJhdWQiOiIzZDIzMzk0OS1hMmZiLTRhYjAtYjdlYy00NmY2MjU1YzUxMGUiLCJpc3MiOiJodHRwczovL2lkZW50aXR5Lm9jdG8uYWkiLCJpYXQiOjE3MTk2Nzg3NTl9.m3RpRr3vLoGekWhvk1fXHlTBAg1fkkO2V2eMfXddjLf30taFXnAOLoJIWHsGLupliLVW3JL9_wxPRhL2cGpDH_1_PeWDUYxKk_ZY1RjL9s_yoJf7fGeEy9-VPbmvjfYXcQTa_sxE99GeqwaODud26e_O04Q9n2KiNSwdeeion6ki9ctpIbDXZ3Wit_ltumGY0axDRInfm_QHwvo92r4W-3g8jN4lskS7ZA7CT5LCRq7GzFs5E2HsZq-59RnVhxnray6nG5fyG3U67Ff0ZCTuRaDY2DqRDOgQw1zRqTYMoW8zeOfozRxlGygdRcRU3AxnUOGOi7yijetkE9I60H9qlg"

# Ingest Data

In [203]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

In [204]:
files = os.listdir("../workout_data")
file_texts = []
for file in files:
    with open(f"../workout_data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1024, chunk_overlap=128, 
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))

In [205]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [206]:
embeddings = HuggingFaceEmbeddings()

In [207]:
vector_store = FAISS.from_documents(
    file_texts,
    embedding=embeddings
)

# Search the Data

In [208]:
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="mixtral-8x7b-instruct",
        max_tokens=8192,
        presence_penalty=0,
        temperature=0,
        top_p=0.95,
        octoai_api_token=OCTOAI_API_TOKEN
    )

/home/cmagorian/Storage/repos/hackathon-0629/.venv/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [209]:
retriever = vector_store.as_retriever()

In [225]:
from langchain.prompts import ChatPromptTemplate
template="""You are a data analyst. You will be given unstructured data texts and figure out the schema. Return the schema. Be as general as possible so the schema can be used on most data input. Return the schema as Pydantic model code. If possible, create multiple tables and relations between them when you see fit. Only output a code segment. Do not provide Sample usage or comments.
Document: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [226]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [232]:
response = chain.invoke("Show me the table schema creation script")
print(response)



 ```python
from typing import List
from pydantic import BaseModel

class Exercise(BaseModel):
    name: str
    sets: int
    reps: int

class Workout(BaseModel):
    day: int
    exercises: List[Exercise]

class Program(BaseModel):
    name: str
    workouts: List[Workout]

# Define the program schema
program = Program(
    name="Gym Workout Program",
    workouts=[
        Workout(
            day=1,
            exercises=[
                Exercise(name="Squats", sets=3, reps=8),
                Exercise(name="Deadlifts", sets=3, reps=8),
                Exercise(name="Lunges", sets=3, reps=10),
                Exercise(name="Leg press", sets=3, reps=10),
                Exercise(name="Calf raises", sets=3, reps=15),
            ]
        ),
        Workout(
            day=2,
            exercises=[
                Exercise(name="Plank", sets=3, reps=30),
                Exercise(name="Bicycle crunches", sets=3, reps=20),
                Exercise(name="Russian twists", sets=3, reps=

In [219]:
from langchain_core.messages import ChatMessage
from octoai.client import OctoAI
from clients.ollm import ChatCompletionResponseFormat
from typing import List
from pydantic import BaseModel

class Exercise(BaseModel):
    name: str
    sets: int
    reps: int

class Workout(BaseModel):
    day: int
    exercises: List[Exercise]

class Program(BaseModel):
    name: str
    workouts: List[Workout]

all_classes = [Exercise, Workout, Program]
client = OctoAI(api_key=OCTOAI_API_TOKEN)
completion = client.text_gen.create_chat_completion(
    model="mistral-7b-instruct",
    messages=[ChatMessage(role="system", content="You are a helpful assistant designed to output JSON."),
                ChatMessage(role="user", content="I bench pressed 95 lbs for 3 sets of 5 reps each today, and I squatted 135 lbs for 3 sets of 10. Please record this.")],
    max_tokens=512,
    presence_penalty=0,
    temperature=0.1,
    top_p=0.9,
    response_format=ChatCompletionResponseFormat(
        type="json_object",
        schema=Workout.model_json_schema(),
    ),
)
print(completion.choices[0].message.content)


{"answer": "I bench pressed 95 lbs for 3 sets of 5 reps each and squatted 135 lbs for 3 sets of 10 reps today."}
